### Assignment 2 : Prediction Challenge
### Keertana Chidambaram (Collaborator: Dhruval Bhatt)

NOTE: all notes are for Dhruval.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
df = pd.read_csv('nlsy training set.csv')

In [3]:
df.head()

,E5011701,E5011702,E5011703,E5011704,E5011705,E5011706,E5011707,E5011708,E5011709,E5011710,...,Z9122600,Z9141400,Z9141500,Z9141600,Z9141700,Z9141800,Z9141900,Z9142000,Z9142100,diag.id
0,-4,-4,2,2,2,1,1,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,6328
1,-4,-4,-4,-4,-4,-4,-4,2,2,2,...,17,7500,0,0,6,0,7500,0,18,388
2,-4,-4,-4,2,2,4,4,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,7655
3,-4,-4,-4,2,2,2,1,1,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,3517
4,-4,-4,-4,-4,-4,-4,2,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,8540


In [6]:
df.shape

(7187, 4886)

Unanswered Qs:
- To include or exclude observations with missing data from the test set while building the model??
- If included, should the model predict the 'type' of the missing data?

New learnings:
- Dealing with missing data: treat as another feature, predict feature (mean/median/mode imputation, KNN imputation), skip feature (?)

Notes: 
- Predictor variables are mostly categorical but predicted variable is ordinal categorical
- Error is MSE, so take predicted variable as continuous 
- Missing Ys are not needed, make an error function that accounts for all this
- Since missing Ys not needed, predict missing Y as a -1. That way even if prediction is wrong, we still get small MSE

Stuff we could do:
- One-hot encoding for categorical data
- Regularization
- Remove missing data from train set

In [22]:
def calc_MSE(y, y_hat):
    '''Calculates the Mean Squared Error given original and predicted values'''
    y_new = y[y>=0]
    y_hat_new = y_hat[y>=0]
    err = sum((y_hat[y>=0] - y[y>=0]) ** 2)
    return err

List of available models:
1. Regression - NO. used when Xs are continuous
2. Classification - NO. used when predicted variables is categorical
3. Logistic regression - NO. used when predicted variable is binary, multi-class logistic when y is categorical
4. Decision trees - works!
5. Random forest - works!
6. Neural networks - works!
7. Ensemble methods - works!

In [26]:
sum(df['U1031900'] >= 0) * 100 / df.shape[0]

49.32517044663977

Using ready-made python packages for now

In [ ]:
# Linear regression
# OLS
import statsmodels.api as sm
xvars = ['bin1', 'bin2','bin3','bin4','bin5']
X = data[xvars]
y = data['coolness']
result = sm.OLS(y, X).fit()

# K Nearest Neighbor
from sklearn import neighbors
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=10)
model = neighbors.KNeighborsClassifier(n_neighbors = 2)
x_vars = ['X1', 'X2', 'X3']
train_data[x_vars]
res = model.fit(train_data[x_vars], train_data['Y'])
print('Predicted value for (1,1,1) is:')
print(res.predict([(0,0,0)]))

# Leave one out cross-validation
Xvars = X.values
yvars = y.values
N_loo = Xvars.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(Xvars)
MSE_vec = np.zeros(N_loo)
y_pred = np.zeros(N_loo)

for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    LogReg = LogisticRegression(solver='newton-cg',multi_class='multinomial').fit(X_train, y_train)
    y_pred[test_index] = LogReg.predict(X_test)
    
    MSE_vec[test_index] = y_test != y_pred[test_index]

MSE_loo = MSE_vec.mean()
print('test estimate MSE loocv=', MSE_loo)

# K-fold CV
Xvars = X.values
yvars = y.values
k = 4
kf = KFold(n_splits=4, shuffle=True, random_state=10)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)
y_pred = np.zeros(len(yvars))

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    LogReg = LogisticRegression(solver='newton-cg',multi_class='multinomial').fit(X_train, y_train)
    
    y_pred[test_index] = LogReg.predict(X_test)

    err = y_pred[test_index] != y_test
    MSE_vec_kf[k_ind] = err.mean()
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
print('test estimate MSE k-fold (k=4) =', MSE_kf)

# Decision trees
xvars = ['female', 'age', 'educ', 'dem', 'rep']
X = data[xvars]
y = data['biden']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=25)
biden_tree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=5)
biden_tree.fit(X_train, y_train)
y_pred = biden_tree.predict(X_test)
MSE1 = mean_squared_error(y_test, y_pred)
print('MSE=', MSE1)

# Search for best tree
param_dist1 = {'max_depth': [3, 10],
               'min_samples_split': sp_randint(2, 20),
               'min_samples_leaf': sp_randint(2, 20)}

biden_tree2 = DecisionTreeRegressor()

random_search1 = RandomizedSearchCV(biden_tree2, 
                                    param_distributions=param_dist1,
                                    n_iter=100, 
                                    n_jobs=-1, 
                                    cv=5, 
                                    random_state=25,
                                    scoring='neg_mean_squared_error')

random_search1.fit(X, y)
print('Best estimator = ', random_search1.best_estimator_)
print('Best tuning parameters = ', random_search1.best_params_)
print('MSE of optimal results =', -random_search1.best_score_)

# Random forest
param_dist2 = { 'n_estimators': [10, 200],
                'max_depth': [3, 10],
                'min_samples_split': sp_randint(2, 20),
                'min_samples_leaf': sp_randint(2, 20),
                'max_features': sp_randint(1, 5)}

biden_tree3 = RandomForestRegressor(bootstrap=True,oob_score=True, random_state=25)

random_search2 = RandomizedSearchCV(biden_tree3, param_distributions=param_dist2, n_iter=100,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')

random_search2.fit(X, y)
print('Best estimator = ', random_search2.best_estimator_)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

# Solution 1.c.
forest = RandomForestClassifier(bootstrap=True, oob_score=True)

param_dist2 = { 'n_estimators': sp_randint(10, 200),
                'max_depth': sp_randint(2, 4),
                'min_samples_split': sp_randint(2, 20),
                'min_samples_leaf': sp_randint(2, 20),
                'max_features': sp_randint(1, 4)}

random_search2 = RandomizedSearchCV(forest, param_distributions=param_dist2, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search2.fit(X, y)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

# Solution 1.d.
svc_model = SVC(kernel='rbf')

param_dist3 = { 'C': sp_uniform(loc=0.1, scale=10.0),
                'gamma': ['scale', 'auto'],
                'shrinking': [True, False]}
random_search3 = RandomizedSearchCV(svc_model, param_distributions=param_dist3, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search3.fit(X, y)
print('Best tuning parameters = ', random_search3.best_params_)
print('MSE of optimal results =', -random_search3.best_score_)

# Solution 1.e.
nn = MLPClassifier(solver='lbfgs')

param_dist4 = {'hidden_layer_sizes': sp_randint(1, 100),
                'activation': ['logistic', 'relu'],
                'alpha': sp_uniform(0.1, 10.0)}

random_search4 = RandomizedSearchCV(nn, param_distributions=param_dist4, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search4.fit(X, y)
print('Best tuning parameters = ', random_search4.best_params_)
print('MSE of optimal results =', -random_search4.best_score_)